# Data preparation

In [1]:
# Import pandas to make DataFrames from the available datasets
import pandas as pd

In [2]:
# Read datasets and assign them in each DataFrame variable
df_product_catalog = pd.read_excel('/Users/sakabumi/project/dsw-2023/Product Catalog.xlsx')
df_product_pos_trx = pd.read_excel('/Users/sakabumi/project/dsw-2023/Product Name from PoS Transactions.xlsx')

In [3]:
df_product_catalog.head()

,Product SKU,Brand,Type,Formula
0,Urea Petro,PIHC,Urea,NaN
1,Urea PIM,PIHC,Urea,NaN
2,Urea Nitrea,PIHC,Urea,NaN
3,Urea Daun Buah,PIHC,Urea,NaN
4,Urea Pusri,PIHC,Urea,NaN


In [4]:
# Checking how many NaN values are there
df_product_catalog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Product SKU  187 non-null    object
 1   Brand        187 non-null    object
 2   Type         187 non-null    object
 3   Formula      123 non-null    object
dtypes: object(4)
memory usage: 6.0+ KB


In [5]:
# Checking how many unique values on each column to help further data exploration
df_product_catalog.describe()

,Product SKU,Brand,Type,Formula
count,187,187,187,123
unique,186,9,10,57
top,ENTEC 13-10-20,PIHC,Majemuk,15-15-15
freq,2,43,123,15


From 187 entries, there are:
- 186 unique product SKUs
- 9 brands,
- 10 types, &
- 57 unique formulas

The occurence of Formula is equal to the frequency of Type "Majemuk". Therefore, Formula is not null only for that Type.

In [6]:
df_product_catalog = df_product_catalog.drop_duplicates()
df_product_catalog = df_product_catalog.reset_index(drop=True)

In [7]:
df_product_catalog.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 0 to 186
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Product SKU  186 non-null    object
 1   Brand        186 non-null    object
 2   Type         186 non-null    object
 3   Formula      122 non-null    object
dtypes: object(4)
memory usage: 7.3+ KB


In [8]:
df_product_catalog.describe()

,Product SKU,Brand,Type,Formula
count,186,186,186,122
unique,186,9,10,57
top,Urea Petro,PIHC,Majemuk,15-15-15
freq,1,43,122,15


In [9]:
df_product_catalog = df_product_catalog.reset_index(drop=True)

In [10]:
df_product_catalog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Product SKU  186 non-null    object
 1   Brand        186 non-null    object
 2   Type         186 non-null    object
 3   Formula      122 non-null    object
dtypes: object(4)
memory usage: 5.9+ KB


In [11]:
df_product_pos_trx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44002 entries, 0 to 44001
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  44001 non-null  object
dtypes: object(1)
memory usage: 343.9+ KB


In [12]:
df_product_pos_trx.describe()

,Product Name
count,44001
unique,44001
top,Pupuk Urea N 46%
freq,1


In [15]:
# Create a new version of df_product_catalog & df_product_pos_trx
product_catalog = df_product_catalog.copy()
product_pos = df_product_pos_trx.copy()

# Rename column names in the new df "product_catalog" & "product_pos"
product_catalog = product_catalog.rename(columns={
    "Product SKU": "product_sku",
    "Brand": "brand",
    "Type": "type",
    "Formula": "formula"
})
product_pos = product_pos.rename(columns={"Product Name": "product_name"})

# Convert all columns' values in "product_catalog" & "product_pos" to uppercase
product_catalog[['product_sku', 'brand', 'type']] = product_catalog[['product_sku', 'brand', 'type']].astype(str).apply(lambda col: col.str.upper())
product_pos['product_name'] = product_pos['product_name'].str.upper()

In [16]:
# Create a new version of df_product_pos_trx
product_pos = df_product_pos_trx.copy()

# Rename column
product_pos = product_pos.rename(columns={"Product Name": "product_name"})

# Convert all columns' values in "product_product_poscatalog" to uppercase
product_pos['product_name'] = product_pos['product_name'].str.upper()

## Preprocess the columns by cleaning the values

In [17]:
import re

# Create a function to do the cleaning
def preprocess_text(text):
    # Trim leading and trailing spaces
    text = text.strip()
    
    # Remove special characters, non-alphanumeric characters, and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove multiple spaces and replace them with a single space
    text = re.sub(r'\s+', ' ', text)
    
    return text

In [18]:
# Cleaning the product names & formulas
preprocessed_product_names = [preprocess_text(str(name)) for name in product_pos['product_name']]
preprocessed_formula = [preprocess_text(str(formula)) for formula in product_catalog['formula']]

# Put back the preprocessed values to their own columns
product_pos['product_name'] = pd.DataFrame(preprocessed_product_names)
product_catalog['formula'] = pd.DataFrame(preprocessed_formula)

In [19]:
product_catalog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   product_sku  186 non-null    object
 1   brand        186 non-null    object
 2   type         186 non-null    object
 3   formula      186 non-null    object
dtypes: object(4)
memory usage: 5.9+ KB


In [20]:
product_catalog.head()

,product_sku,brand,type,formula
0,UREA PETRO,PIHC,UREA,nan
1,UREA PIM,PIHC,UREA,nan
2,UREA NITREA,PIHC,UREA,nan
3,UREA DAUN BUAH,PIHC,UREA,nan
4,UREA PUSRI,PIHC,UREA,nan


In [21]:
product_catalog.describe()

,product_sku,brand,type,formula
count,186,186,186,186
unique,186,9,10,58
top,UREA PETRO,PIHC,MAJEMUK,nan
freq,1,43,122,64


In `formula`, the count increases to 186 due to the rise of `nan` after the data cleaning in that column.
The non-nan values can be used to tag the product names as type `MAJEMUK` as long as the product names contain the formula.

In [22]:
# Dataset from the PoS is joined by product SKUs data from
# dataset `product_catalog` using join keys `product_name` & `product_sku`.
product_pos = product_pos.merge(
    product_catalog['product_sku'],
    left_on='product_name',
    right_on='product_sku',
    how='left')

In [23]:
product_pos.head()

,product_name,product_sku
0,PUPUK UREA N 46,NaN
1,PUPUK AMONIUM SULFAT ZA,NaN
2,PUPUK SUPER FOSFAT SP36,NaN
3,PUPUK NPK PHONSKA,NaN
4,PUPUK NPK FORMULA KHUSUS,NaN


In [24]:
product_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44002 entries, 0 to 44001
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  44002 non-null  object
 1   product_sku   39 non-null     object
dtypes: object(2)
memory usage: 687.7+ KB


Turns out, there are only 39 out of 44,002 product names which can be mapped.

# Mapping

In [25]:
list_product_type = list(product_catalog['type'].unique())
list_formula = list(product_catalog['formula'].unique())

In [26]:
list_product_type

['UREA',
 'NITROGEN',
 'ZA',
 'ZK',
 'MIKRO',
 'FOSFAT',
 'ORGANIK',
 'MAJEMUK',
 'KALIUM',
 'MG']

In [27]:
list_formula

['nan',
 '151515',
 '121217',
 '12622',
 '16168',
 '20200',
 '16200',
 '161616',
 '15920',
 '3068',
 '281010',
 '201010',
 '201018',
 '13627',
 '181014',
 '121120',
 '131324',
 '92525',
 '15150',
 '12610',
 '05234',
 '8939',
 '15156',
 '13827',
 '7635',
 '151022',
 '21147',
 '18614',
 '18810',
 '201012',
 '28613',
 '81519',
 '9156',
 '12624',
 '05232',
 '15015',
 '12600',
 '121236',
 '181818',
 '61828',
 '01617',
 '13046',
 '1370463',
 '131121',
 '2577',
 '121118',
 '18126',
 '19919',
 '05035',
 '18460',
 '7634',
 '161018',
 '131020',
 '15520',
 '131111',
 '151020',
 '15200',
 '20614']

In [28]:
list_formula.remove('nan')

In [29]:
# Create a function to map product type to product name
def map_product_type(product_name):
    for product_type in list_product_type:
        if product_type in product_name:
            return product_type
    return "UNKNOWN"

# Create a function to map product type `Majemuk` to product name which contains a certain formula
def map_product_type_majemuk(product_name):
    for formula in list_formula:
        if formula in product_name:
            return "MAJEMUK"
    return "UNKNOWN"

In [30]:
# Mapping process using function `map_product_type()` & `map_product_type_majemuk()`
mapped_product_types = [map_product_type(str(name)) for name in product_pos['product_name']]
mapped_product_types_majemuk = [map_product_type_majemuk(str(name)) for name in product_pos['product_name']]

# Convert the list of the mapped product types to a DataFrame column within product_pos
product_pos['product_type_map'] = pd.DataFrame(mapped_product_types)
product_pos['product_type_map_majemuk'] = pd.DataFrame(mapped_product_types_majemuk)

In [31]:
product_pos.head()

,product_name,product_sku,product_type_map,product_type_map_majemuk
0,PUPUK UREA N 46,NaN,UREA,UNKNOWN
1,PUPUK AMONIUM SULFAT ZA,NaN,ZA,UNKNOWN
2,PUPUK SUPER FOSFAT SP36,NaN,FOSFAT,UNKNOWN
3,PUPUK NPK PHONSKA,NaN,UNKNOWN,UNKNOWN
4,PUPUK NPK FORMULA KHUSUS,NaN,UNKNOWN,UNKNOWN


In [32]:
product_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44002 entries, 0 to 44001
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   product_name              44002 non-null  object
 1   product_sku               39 non-null     object
 2   product_type_map          44002 non-null  object
 3   product_type_map_majemuk  44002 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


In [33]:
product_pos.groupby(['product_type_map'])['product_name'].count()

product_type_map
FOSFAT         11
KALIUM         22
MAJEMUK        18
MG             68
MIKRO          37
NITROGEN        4
ORGANIK        44
UNKNOWN     42371
UREA          727
ZA            656
ZK             44
Name: product_name, dtype: int64

There are 42371 product names with no `product_type`

In [34]:
product_pos.groupby(['product_type_map_majemuk'])['product_name'].count()

product_type_map_majemuk
MAJEMUK     1143
UNKNOWN    42859
Name: product_name, dtype: int64

Most of the `product_name`s are not mapped still --> 42371 out of 44,002

In [35]:
import numpy as np

# Replace the values in "product_type_map" 
product_pos['product_type_map'] = np.where(
    product_pos['product_type_map_majemuk'] != 'UNKNOWN',
    product_pos['product_type_map_majemuk'],
    product_pos['product_type_map']
)

In [36]:
# Replace the values in "product_type_map" 
product_pos['product_type_map'] = np.where(
    product_pos['product_type_map_majemuk'] != 'UNKNOWN',
    product_pos['product_type_map_majemuk'],
    product_pos['product_type_map']
)

In [37]:
product_pos[product_pos['product_type_map_majemuk'] == 'MAJEMUK'].head()

,product_name,product_sku,product_type_map,product_type_map_majemuk
730,NPK PELANGI 151515,NaN,MAJEMUK,MAJEMUK
731,NPK PELANGI 161616,NaN,MAJEMUK,MAJEMUK
732,NPK PELANGI 1212172,NaN,MAJEMUK,MAJEMUK
733,NPK PELANGI 136274B,NaN,MAJEMUK,MAJEMUK
1001,NPK 161616 PAK TANI,NaN,MAJEMUK,MAJEMUK


In [38]:
product_pos.groupby(['product_type_map'])['product_name'].count()

product_type_map
FOSFAT         11
KALIUM         22
MAJEMUK      1153
MG             16
MIKRO          35
NITROGEN        4
ORGANIK        44
UNKNOWN     41292
UREA          725
ZA            656
ZK             44
Name: product_name, dtype: int64

In [39]:
list_brand = list(product_catalog['brand'].unique())

In [40]:
list_brand

['PIHC',
 'MUTIARA',
 'MAHKOTA',
 'PAK TANI',
 'YARA',
 'TAWON',
 'DGW/HEXTAR',
 'BASF',
 'LAOYING']

In [41]:
# Create a function to map brand to product name
def map_brand(product_name):
    for brand in list_brand:
        if brand in product_name:
            return brand
    return "UNKNOWN"

In [42]:
# Mapping process using function `map_brand()`
# Convert argument `name` within the function `map_brand()` to avoid this error:
#   TypeError: argument of type 'float' is not iterable

mapped_brands = [map_brand(str(name)) for name in product_pos['product_name']]

# Convert the list of the mapped brands to a DataFrame column within product_pos
product_pos['brand_map'] = pd.DataFrame(mapped_brands)

In [43]:
product_pos.groupby(['brand_map'])['product_name'].count()

brand_map
BASF            8
LAOYING        45
MAHKOTA       149
MUTIARA       404
PAK TANI      252
TAWON         273
UNKNOWN     42610
YARA          261
Name: product_name, dtype: int64

In [44]:
product_pos[product_pos['brand_map'] != 'UNKNOWN'].head()

,product_name,product_sku,product_type_map,product_type_map_majemuk,brand_map
81,KCL MAHKOTA,NaN,UNKNOWN,UNKNOWN,MAHKOTA
82,MUTIARA,NaN,UNKNOWN,UNKNOWN,MUTIARA
83,MUTIARA GROWER,NaN,UNKNOWN,UNKNOWN,MUTIARA
366,ZA TAWON50KG,NaN,ZA,UNKNOWN,TAWON
410,DAP TAWON,NaN,UNKNOWN,UNKNOWN,TAWON


In [45]:
product_pos = product_pos.drop(['product_type_map_majemuk'], axis=1)

In [46]:
product_pos[product_pos['brand_map'] != 'UNKNOWN'].head()

,product_name,product_sku,product_type_map,brand_map
81,KCL MAHKOTA,NaN,UNKNOWN,MAHKOTA
82,MUTIARA,NaN,UNKNOWN,MUTIARA
83,MUTIARA GROWER,NaN,UNKNOWN,MUTIARA
366,ZA TAWON50KG,NaN,ZA,TAWON
410,DAP TAWON,NaN,UNKNOWN,TAWON


In [47]:
product_pos[product_pos['product_type_map'] != 'UNKNOWN'].count()

product_name        2710
product_sku           22
product_type_map    2710
brand_map           2710
dtype: int64

In [48]:
product_pos[product_pos['brand_map'] != 'UNKNOWN'].count()

product_name        1392
product_sku            2
product_type_map    1392
brand_map           1392
dtype: int64

In [49]:
product_pos.groupby(['product_type_map'])['product_name'].count()

product_type_map
FOSFAT         11
KALIUM         22
MAJEMUK      1153
MG             16
MIKRO          35
NITROGEN        4
ORGANIK        44
UNKNOWN     41292
UREA          725
ZA            656
ZK             44
Name: product_name, dtype: int64

In [50]:
product_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44002 entries, 0 to 44001
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_name      44002 non-null  object
 1   product_sku       39 non-null     object
 2   product_type_map  44002 non-null  object
 3   brand_map         44002 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


Create a new column which concatenates `product_name`, `product_type`, & `product_brand`

In [51]:
product_pos['product_name_type_brand'] = product_pos['product_name'] + '-' + product_pos['product_type_map'] + '-' + product_pos['brand_map']

Trial, creating the model

In [52]:
product_pos[product_pos['product_sku'].notnull()].head()

,product_name,product_sku,product_type_map,brand_map,product_name_type_brand
415,MESTAC,MESTAC,UNKNOWN,UNKNOWN,MESTAC-UNKNOWN-UNKNOWN
504,FERTIPHOS,FERTIPHOS,UNKNOWN,UNKNOWN,FERTIPHOS-UNKNOWN-UNKNOWN
637,UREA PETRO,UREA PETRO,UREA,UNKNOWN,UREA PETRO-UREA-UNKNOWN
734,ZA PETRO,ZA PETRO,ZA,UNKNOWN,ZA PETRO-ZA-UNKNOWN
740,MESTIKALI,MESTIKALI,UNKNOWN,UNKNOWN,MESTIKALI-UNKNOWN-UNKNOWN


In [53]:
product_pos[product_pos['product_sku'].notnull()].info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 415 to 36835
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   product_name             39 non-null     object
 1   product_sku              39 non-null     object
 2   product_type_map         39 non-null     object
 3   brand_map                39 non-null     object
 4   product_name_type_brand  39 non-null     object
dtypes: object(5)
memory usage: 1.8+ KB


In [54]:
product_pos[product_pos['product_sku'].notnull()].describe()

,product_name,product_sku,product_type_map,brand_map,product_name_type_brand
count,39,39,39,39,39
unique,23,23,4,3,23
top,UREA PETRO,UREA PETRO,UNKNOWN,UNKNOWN,UREA PETRO-UREA-UNKNOWN
freq,3,3,17,37,3


In [57]:
product_pos.head()

,product_name,product_sku,product_type_map,brand_map,product_name_type_brand
0,PUPUK UREA N 46,NaN,UREA,UNKNOWN,PUPUK UREA N 46-UREA-UNKNOWN
1,PUPUK AMONIUM SULFAT ZA,NaN,ZA,UNKNOWN,PUPUK AMONIUM SULFAT ZA-ZA-UNKNOWN
2,PUPUK SUPER FOSFAT SP36,NaN,FOSFAT,UNKNOWN,PUPUK SUPER FOSFAT SP36-FOSFAT-UNKNOWN
3,PUPUK NPK PHONSKA,NaN,UNKNOWN,UNKNOWN,PUPUK NPK PHONSKA-UNKNOWN-UNKNOWN
4,PUPUK NPK FORMULA KHUSUS,NaN,UNKNOWN,UNKNOWN,PUPUK NPK FORMULA KHUSUS-UNKNOWN-UNKNOWN


In [60]:
df = product_pos[product_pos['product_sku'].notnull()]

In [62]:
df.head(10)

,product_name,product_sku,product_type_map,brand_map,product_name_type_brand
415,MESTAC,MESTAC,UNKNOWN,UNKNOWN,MESTAC-UNKNOWN-UNKNOWN
504,FERTIPHOS,FERTIPHOS,UNKNOWN,UNKNOWN,FERTIPHOS-UNKNOWN-UNKNOWN
637,UREA PETRO,UREA PETRO,UREA,UNKNOWN,UREA PETRO-UREA-UNKNOWN
734,ZA PETRO,ZA PETRO,ZA,UNKNOWN,ZA PETRO-ZA-UNKNOWN
740,MESTIKALI,MESTIKALI,UNKNOWN,UNKNOWN,MESTIKALI-UNKNOWN-UNKNOWN
4832,NITRALITE,NITRALITE,UNKNOWN,UNKNOWN,NITRALITE-UNKNOWN-UNKNOWN
4906,UREA PUSRI,UREA PUSRI,UREA,UNKNOWN,UREA PUSRI-UREA-UNKNOWN
4953,ZK PETRO,ZK PETRO,ZK,UNKNOWN,ZK PETRO-ZK-UNKNOWN
5138,ZA PAK TANI,ZA PAK TANI,ZA,PAK TANI,ZA PAK TANI-ZA-PAK TANI
5439,UREA PETRO,UREA PETRO,UREA,UNKNOWN,UREA PETRO-UREA-UNKNOWN


In [64]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn import metrics

In [81]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['product_name_type_brand'], df['product_sku'], test_size=0.2, random_state=42)

# Create a text classification pipeline
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
print("Accuracy:", metrics.accuracy_score(y_test, predictions))
print("Classification Report:\n", metrics.classification_report(y_test, predictions))

Accuracy: 0.125
Classification Report:
                          precision    recall  f1-score   support

KAPUR PERTANIAN KEBOMAS       0.00      0.00      0.00         1
      MAGNESIUM SULFATE       0.00      0.00      0.00         1
              MESTIKALI       0.00      0.00      0.00         1
        PETRO BIOFERTIL       0.00      0.00      0.00         1
              PHOSGREEN       0.00      0.00      0.00         1
                PHOSGRO       0.00      0.00      0.00         1
         UREA DAUN BUAH       1.00      1.00      1.00         1
             UREA PETRO       0.00      0.00      0.00         0
             UREA PUSRI       0.00      0.00      0.00         1

               accuracy                           0.12         8
              macro avg       0.11      0.11      0.11         8
           weighted avg       0.12      0.12      0.12         8



/Users/sakabumi/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sakabumi/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sakabumi/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sakabumi/miniconda3/

In [72]:
predictions

array(['UREA PETRO', 'UREA PETRO', 'UREA PETRO', 'UREA DAUN BUAH',
       'UREA PETRO', 'UREA PETRO', 'UREA PETRO', 'UREA PETRO'],
      dtype='<U23')

In [73]:
model.predict(df['product_name_type_brand'])

array(['UREA PETRO', 'FERTIPHOS', 'UREA PETRO', 'ZA PETRO', 'UREA PETRO',
       'NITRALITE', 'UREA PETRO', 'ZK PETRO', 'ZA PETRO', 'UREA PETRO',
       'ZA PETRO', 'UREA PETRO', 'NITRALITE', 'UREA DAUN BUAH',
       'UREA PETRO', 'UREA DAUN BUAH', 'UREA PETRO', 'UREA PETRO',
       'UREA PETRO', 'UREA PETRO', 'ZA PETRO', 'UREA PETRO', 'UREA PETRO',
       'ZK PETRO', 'ZA PETRO', 'FERTIPHOS', 'UREA PETRO', 'UREA PETRO',
       'UREA PETRO', 'UREA PETRO', 'UREA PETRO', 'ZA PETRO', 'UREA PETRO',
       'UREA PETRO', 'ZA PETRO', 'UREA DAUN BUAH', 'UREA PETRO',
       'ZA PETRO', 'UREA PETRO'], dtype='<U23')

In [74]:
model.predict(product_pos['product_name_type_brand'])

array(['UREA PETRO', 'ZA PETRO', 'UREA PETRO', ..., 'UREA PETRO',
       'UREA PETRO', 'UREA PETRO'], dtype='<U23')

In [75]:
product_pos['product_sku_test'] = model.predict(product_pos['product_name_type_brand'])

In [2]:
product_pos.head(10)

NameError: name 'product_pos' is not defined

### Training model to map product names to product types

In [1]:
df_map_types = product_pos[product_pos['product_type_map'].not_null()]

NameError: name 'product_pos' is not defined

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['product_name_type_brand'], df['product_sku'], test_size=0.2, random_state=42)

# Create a text classification pipeline
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
print("Accuracy:", metrics.accuracy_score(y_test, predictions))
print("Classification Report:\n", metrics.classification_report(y_test, predictions))

Feature Extraction

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [62]:
product_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44002 entries, 0 to 44001
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   product_name              44002 non-null  object
 1   product_sku               41 non-null     object
 2   product_type_map          44002 non-null  object
 3   product_type_map_majemuk  44002 non-null  object
 4   brand_map                 44002 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


In [63]:
# Convert text data into numerical vectors
product_names = list(product_pos['product_name'])
product_types = list(product_pos['product_type_map'])
product_brands = list(product_pos['brand_map'])

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Create TF-IDF vectors for product names, product types, and brands
product_names_vectors = tfidf_vectorizer.fit_transform(product_names)
product_types_vectors = tfidf_vectorizer.fit_transform(product_types)
product_brands_vectors = tfidf_vectorizer.fit_transform(product_brands)

In [64]:
from sklearn.metrics.pairwise import cosine_similarity

In [65]:
# Calculate cosine similarity between product names and product types
product_name_type_similarity = cosine_similarity(product_names_vectors, product_types_vectors)

# Calculate cosine similarity between product names and brands
product_name_brand_similarity = cosine_similarity(product_names_vectors, product_brands_vectors)

# Combine similarity scores
combined_similarity = product_name_type_similarity + product_name_brand_similarity

# Define a similarity threshold
similarity_threshold = 1.0  # Adjust as needed

# Map product names to product SKUs using the combined similarity scores
mapped_product_skus = {}

for i, product_name in enumerate(product_names):
    potential_skus = [sku for sku, score in enumerate(combined_similarity[i]) if score > similarity_threshold]
    if potential_skus:
        mapped_product_skus[product_name] = potential_skus

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 16311 while Y.shape[1] == 11

In [66]:
product_pos.head()

,product_name,product_sku,product_type_map,product_type_map_majemuk,brand_map
0,PUPUK UREA N 46,NaN,UREA,UNKNOWN,UNKNOWN
1,PUPUK AMONIUM SULFAT ZA,NaN,ZA,UNKNOWN,UNKNOWN
2,PUPUK SUPER FOSFAT SP36,NaN,FOSFAT,UNKNOWN,UNKNOWN
3,PUPUK NPK PHONSKA,NaN,UNKNOWN,UNKNOWN,UNKNOWN
4,PUPUK NPK FORMULA KHUSUS,NaN,UNKNOWN,UNKNOWN,UNKNOWN


In [67]:
product_pos[product_pos['product_sku'].notnull()].head()

,product_name,product_sku,product_type_map,product_type_map_majemuk,brand_map
109,PETROCAS,PETRO-CAS,UNKNOWN,UNKNOWN,UNKNOWN
415,MESTAC,MESTAC,UNKNOWN,UNKNOWN,UNKNOWN
504,FERTIPHOS,FERTIPHOS,UNKNOWN,UNKNOWN,UNKNOWN
637,UREA PETRO,UREA PETRO,UREA,UNKNOWN,UNKNOWN
734,ZA PETRO,ZA PETRO,ZA,UNKNOWN,UNKNOWN


In [73]:
product_pos = product_pos.drop('product_type_map_majemuk', axis=1)

In [78]:
# product_pos[
#     (product_pos['product_sku'].notnull())
#     & (product_pos['product_type_map'] != 'UNKNOWN')
#     & (product_pos['brand_map'] != 'UNKNOWN')
#     ]

product_pos['product_name_type_brand'] = product_pos['product_name'] + '-' + product_pos['product_type_map'] + '-' + product_pos['brand_map']

In [79]:
product_pos.head()

,product_name,product_sku,product_type_map,brand_map,product_name_type_brand
0,PUPUK UREA N 46,NaN,UREA,UNKNOWN,PUPUK UREA N 46-UREA-UNKNOWN
1,PUPUK AMONIUM SULFAT ZA,NaN,ZA,UNKNOWN,PUPUK AMONIUM SULFAT ZA-ZA-UNKNOWN
2,PUPUK SUPER FOSFAT SP36,NaN,FOSFAT,UNKNOWN,PUPUK SUPER FOSFAT SP36-FOSFAT-UNKNOWN
3,PUPUK NPK PHONSKA,NaN,UNKNOWN,UNKNOWN,PUPUK NPK PHONSKA-UNKNOWN-UNKNOWN
4,PUPUK NPK FORMULA KHUSUS,NaN,UNKNOWN,UNKNOWN,PUPUK NPK FORMULA KHUSUS-UNKNOWN-UNKNOWN


In [83]:
product_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44002 entries, 0 to 44001
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   product_name             44002 non-null  object
 1   product_sku              41 non-null     object
 2   product_type_map         44002 non-null  object
 3   brand_map                44002 non-null  object
 4   product_name_type_brand  44002 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


In [86]:
product_pos = product_pos.fillna('UNKNOWN')

In [87]:
product_pos.head()

,product_name,product_sku,product_type_map,brand_map,product_name_type_brand
0,PUPUK UREA N 46,UNKNOWN,UREA,UNKNOWN,PUPUK UREA N 46-UREA-UNKNOWN
1,PUPUK AMONIUM SULFAT ZA,UNKNOWN,ZA,UNKNOWN,PUPUK AMONIUM SULFAT ZA-ZA-UNKNOWN
2,PUPUK SUPER FOSFAT SP36,UNKNOWN,FOSFAT,UNKNOWN,PUPUK SUPER FOSFAT SP36-FOSFAT-UNKNOWN
3,PUPUK NPK PHONSKA,UNKNOWN,UNKNOWN,UNKNOWN,PUPUK NPK PHONSKA-UNKNOWN-UNKNOWN
4,PUPUK NPK FORMULA KHUSUS,UNKNOWN,UNKNOWN,UNKNOWN,PUPUK NPK FORMULA KHUSUS-UNKNOWN-UNKNOWN


In [88]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn import metrics

In [90]:
product_pos_test = product_pos[product_pos['product_sku'] != 'UNKNOWN']

In [91]:
product_pos_test.head()

,product_name,product_sku,product_type_map,brand_map,product_name_type_brand
109,PETROCAS,PETRO-CAS,UNKNOWN,UNKNOWN,PETROCAS-UNKNOWN-UNKNOWN
415,MESTAC,MESTAC,UNKNOWN,UNKNOWN,MESTAC-UNKNOWN-UNKNOWN
504,FERTIPHOS,FERTIPHOS,UNKNOWN,UNKNOWN,FERTIPHOS-UNKNOWN-UNKNOWN
637,UREA PETRO,UREA PETRO,UREA,UNKNOWN,UREA PETRO-UREA-UNKNOWN
734,ZA PETRO,ZA PETRO,ZA,UNKNOWN,ZA PETRO-ZA-UNKNOWN


In [93]:
product_pos_test['product_sku'] = np.where(
    product_pos_test['product_sku'] == 'UNKNOWN',
    product_pos_test['product_sku'] == '',
    product_pos_test['product_sku']
)

<ipython-input-93-15026b2656b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_pos_test['product_sku'] = np.where(


In [94]:
product_pos.head()

,product_name,product_sku,product_type_map,brand_map,product_name_type_brand
0,PUPUK UREA N 46,UNKNOWN,UREA,UNKNOWN,PUPUK UREA N 46-UREA-UNKNOWN
1,PUPUK AMONIUM SULFAT ZA,UNKNOWN,ZA,UNKNOWN,PUPUK AMONIUM SULFAT ZA-ZA-UNKNOWN
2,PUPUK SUPER FOSFAT SP36,UNKNOWN,FOSFAT,UNKNOWN,PUPUK SUPER FOSFAT SP36-FOSFAT-UNKNOWN
3,PUPUK NPK PHONSKA,UNKNOWN,UNKNOWN,UNKNOWN,PUPUK NPK PHONSKA-UNKNOWN-UNKNOWN
4,PUPUK NPK FORMULA KHUSUS,UNKNOWN,UNKNOWN,UNKNOWN,PUPUK NPK FORMULA KHUSUS-UNKNOWN-UNKNOWN


In [92]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(product_pos['product_name_type_brand'], product_pos['product_sku'], test_size=0.4, random_state=42)

# Create a text classification pipeline
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
print("Accuracy:", metrics.accuracy_score(y_test, predictions))
print("Classification Report:\n", metrics.classification_report(y_test, predictions))

Accuracy: 0.999090960740867


/Users/sakabumi/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sakabumi/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report:
                             precision    recall  f1-score   support

   KAPUR PERTANIAN KEBOMAS       0.00      0.00      0.00         1
                  MEROKEZA       0.00      0.00      0.00         1
                 MESTIKALI       0.00      0.00      0.00         1
MUTIARA GROWER 15-09-20+TE       0.00      0.00      0.00         1
                 NITRALITE       0.00      0.00      0.00         1
         NPK KUJANG 30-6-8       0.00      0.00      0.00         1
          PELANGI 15-15-15       0.00      0.00      0.00         1
          PELANGI 16-16-16       0.00      0.00      0.00         1
     PETRO NITRAT 16-16-16       0.00      0.00      0.00         1
                 PETRO-CAS       0.00      0.00      0.00         1
                   PHOSGRO       0.00      0.00      0.00         1
                   UNKNOWN       1.00      1.00      1.00     17585
                   URECOTE       0.00      0.00      0.00         1
               ZA PAK T

/Users/sakabumi/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
